# Primer Parcial

#### 01-4900 | 2C 2025

*Alumnos* :
*   MARTINEZ CANNELLA, IÑAKI
*   GHIANO, GONZALO AGUSTÍN
*   CHAILE, FACUNDO MARTIN
*   ALAZRAKI, MICAELA AGUSTINA
*   RINAUDO, DIEGO NAHUEL

*Grupo* :
Data y Familia

## Enunciado

La empresa “Business Prop SRL” contrata nuestros servicios para que le desarrollemos un
modelo que permita predecir si los departamentos vendidos pagan o no comisión, cuando su
precio de venta sea superior a un determinado valor.
Para ello, nos comparten un dataset llamado dptos_entrenamiento.csv, que contiene
información de departamentos vendidos en distintos lugares de Argentina y el exterior. Este
dataset será el que utilicemos para el entrenamiento del modelo construido.
El dataset de predicción a utilizar es dptos_predecir.csv, el cual no contiene la etiqueta de la
variable clase (por defecto viene indicada como “no paga”).

## Solución

### Carga de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Agregamos modelos de ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import make_scorer

import os


### Lectura de los dataset

Cargamos los DataSets, que se encuentran en formato CSV, para poder armar los DataFrames correspondientes.

In [ ]:
# guardamos las url en variables
url_train = 'https://raw.githubusercontent.com/pokengineer/DataScience/refs/heads/main/assessment/dptos_entrenamiento.csv'
url_test = 'https://raw.githubusercontent.com/pokengineer/DataScience/refs/heads/main/assessment/dptos_predecir.csv'

# leemos los datos
df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)

# mostramos para tener una idea de como se ven los datos
print("Datos de Entrenamiento:", df_train.shape)
df_train.head()

In [ ]:
print("Datos de Test:", df_test.shape)
df_test.head()

### Análisis Exploratorio

#### Variables categóricas y numéricas

Entendemos los datos antes de intentar modelar:
* Buscamos estadísticas generales para detectar diferencias entre variables con escalas distintas, columnas con valores faltantes, repetidos o sin variación e incluso posibles datos. 
* Detectamos cuales son las variables categóricas y cuales las numéricas

In [ ]:
# mostramos estadísticas generales de los datos de training
print("Estadísticas Generales de los Datos de Entrenamiento:")
display(df_train.describe(include='all').transpose())

# identificamos variables categóricas y numéricas
categorical_cols = df_train.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df_train.select_dtypes(include=['number']).columns.tolist()

# printeamos las variables categóricas y numéricas
print("\nVariables Categóricas:", categorical_cols)
print("\nVariables Numéricas:", numerical_cols)


# verificamos variación en las columnas de los datos de entrenamiento
print("\nVariación en Columnas de Datos de Entrenamiento:")
for col in df_train.columns:
    unique_values = df_train[col].nunique()
    print(f"{col}: {unique_values} valores únicos")

### Valores faltantes o nulos

* Obtenemos el número de nulos existentes en los datasets
* Si alguna columna presenta nulos, debemos saber como analizar según el tipo de variable y el contexto. Por ejemplo, si es numérica debemos reemplazar con 0 o el avg; si es categórica, podemos reemplazar el nulo con un dato significativo

In [ ]:
missing_counts = df_train.isnull().sum().sort_values(ascending=False)
missing_perc = (missing_counts / df_train.shape[0]) * 100

df_missing = pd.DataFrame({
    "nulos": missing_counts,
    "pct_nulos": missing_perc.round(2)
}).loc[missing_perc > 0]  # solo se muestran las columnas con > 0% nulos

print("Columnas con valores faltantes:")
display(df_missing)

In [ ]:
# evaluamos el impacto de eliminar filas con nulos

print("Cantidad total de filas:", df_train.shape[0])
print("Cantidad de filas sin ningún nulo:", df_train.dropna().shape[0])
print("Porcentaje de filas que conservarías:", round(100 * df_train.dropna().shape[0] / df_train.shape[0], 2), "%")

In [ ]:
# según el resultado anterior, vemos que todas las filas del dataset tienen al menos un valor nulo, por lo que eliminar filas no es una opción viable. 
# si hacemos dropna() perderíamos todo el dataset.

# vemos cuantos nulos hay por columna
total_filas = df_train.shape[0]
nulos_por_col = df_train.isnull().sum().sort_values(ascending=False)
porcentaje_nulos = (nulos_por_col / total_filas)

nulos_df = pd.DataFrame({
    'Nulos': nulos_por_col,
    'Porcentaje': porcentaje_nulos.round(2)
})

display(nulos_df.head(20))  # solo nos quedamos con las primeras 20 filas con nulos

### Eliminación de columnas con muchos nulos (usando un umbral)

In [ ]:
umbral_nulos = 0.8 ## preguntar!!

# identificamos las columnas que superan el umbral de nulos
cols_muchos_nulos = porcentaje_nulos[porcentaje_nulos > umbral_nulos].index.tolist()

print("Columnas con más del % de nulos establecido (", umbral_nulos*100, "%):")
print(cols_muchos_nulos)

# eliminamos las columnas con muchos nulos
df_limpio = df_train.drop(columns=cols_muchos_nulos)

print("\nDataset reducido de forma que se eliminaron las columnas con muchos nulos:")
print("Forma original:", df_train.shape)    
print("Forma reducida:", df_limpio.shape)

###### chequeo ######

# vemos el porcentaje total de celdas nulas en el dataset limpio
display(df_limpio.isnull().mean().sort_values(ascending=True).head(20))
